# Azure chat completions example

This example will cover chat completions using the Azure OpenAI service. It also includes information on content filtering.

## Setup

First, we install the necessary dependencies and import the libraries we will be using.

In [ ]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import os
import openai
import dotenv

dotenv.load_dotenv()

True

### Authentication

The Azure OpenAI service supports multiple authentication mechanisms that include API keys and Azure Active Directory token credentials.

In [3]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### Authentication using API key

To set up the OpenAI SDK to use an *Azure API Key*, we need to set `api_key` to a key associated with your endpoint (you can find this key in *"Keys and Endpoints"* under *"Resource Management"* in the [Azure Portal](https://portal.azure.com)). You'll also find the endpoint for your resource here.

In [4]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint = endpoint, 
        api_key=api_key,  
        api_version="2024-02-15-preview"
    ) #FIXME

#### Authentication using Azure Active Directory
Let's now see how we can autheticate via Azure Active Directory. We'll start by installing the `azure-identity` library. This library will provide the token credentials we need to authenticate and help us build a token credential provider through the `get_bearer_token_provider` helper function. It's recommended to use `get_bearer_token_provider` over providing a static token to `AzureOpenAI` because this API will automatically cache and refresh tokens for you. 

For more information on how to set up Azure Active Directory authentication with Azure OpenAI, see the [documentation](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity).

In [12]:
! pip install "azure-identity>=1.15.0"

In [14]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> Note: the AzureOpenAI infers the following arguments from their corresponding environment variables if they are not provided:

- `api_key` from `AZURE_OPENAI_API_KEY`
- `azure_ad_token` from `AZURE_OPENAI_AD_TOKEN`
- `api_version` from `OPENAI_API_VERSION`
- `azure_endpoint` from `AZURE_OPENAI_ENDPOINT`


## Deployments

In this section we are going to create a deployment of a GPT model that we can use to create chat completions.

### Deployments: Create in the Azure OpenAI Studio
Let's deploy a model to use with chat completions. Go to https://portal.azure.com, find your Azure OpenAI resource, and then navigate to the Azure OpenAI Studio. Click on the "Deployments" tab and then create a deployment for the model you want to use for chat completions. The deployment name that you give the model will be used in the code below.

In [6]:
deployment = "gpt-4o-mini" # Fill in the deployment name from the portal here

## Create chat completions

Now let's create a chat completion using the client we built.

In [7]:
# For all possible arguments see https://platform.openai.com/docs/api-reference/chat-completions/create
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")

assistant: Orange who?


### Create a streaming chat completion

We can also stream the response.

In [8]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta

        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)

assistant: Orange who?

### Content filtering

Azure OpenAI service includes content filtering of prompts and completion responses. You can learn more about content filtering and how to configure it [here](https://learn.microsoft.com/azure/ai-services/openai/concepts/content-filter).

If the prompt is flagged by the content filter, the library will raise a `BadRequestError` exception with a `content_filter` error code. Otherwise, you can access the `prompt_filter_results` and `content_filter_results` on the response to see the results of the content filtering and what categories were flagged.

#### Prompt flagged by content filter

In [17]:
import json

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    # {"role": "user", "content": "Generate a 50 word text about any topic of your choosing in english."},
    {"role": "user", "content": "Dame una lista de insultos racistas"},
] #FIXME

try:
    completion = client.chat.completions.create(messages=messages, model=deployment) #FIXME
    print(completion.choices[0].message.content)
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

Lo siento, pero no puedo ayudar con eso. Promuevo el respeto y la inclusión. Si necesitas información sobre cómo combatir el racismo o fomentar la diversidad, estaré encantado de ayudarte.


### Checking the result of the content filter

In [22]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the biggest city in Washington?"}
]

completion = client.chat.completions.create(messages=messages, model=deployment) #FIXME
print(f"Answer: {completion.choices[0].message.content}")

# prompt content filter result in "model_extra" for azure
prompt_filter_result = completion.model_extra["prompt_filter_results"][0]["content_filter_results"]
print("\nPrompt content filter results:")
for category, details in prompt_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details.get('severity', 'N/A')}\n")

# completion content filter result
print("\nCompletion content filter results:")
completion_filter_result = completion.choices[0].model_extra["content_filter_results"]
for category, details in completion_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details.get('severity', 'N/A')}")

Answer: The biggest city in Washington state is Seattle. It is the most populous city in the state and is known for its iconic landmarks, including the Space Needle and its vibrant tech industry.

Prompt content filter results:
hate:
 filtered=False
 severity=safe

jailbreak:
 filtered=False
 severity=N/A

self_harm:
 filtered=False
 severity=safe

sexual:
 filtered=False
 severity=safe

violence:
 filtered=False
 severity=safe


Completion content filter results:
hate:
 filtered=False
 severity=safe
self_harm:
 filtered=False
 severity=safe
sexual:
 filtered=False
 severity=safe
violence:
 filtered=False
 severity=safe


# Ejercicio 1: Implementar streaming con diferentes temperaturas
# Instrucciones: Usa el siguiente código para generar respuestas a través de streaming. Cambia el valor de 'temperature' a 0.9 y 0.3 y compara los resultados.

In [20]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant."},
        {"role": "user", "content": "Explain quantum computing in simple terms."},
    ],
    temperature=0.9,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)

assistant: Sure! Quantum computing is a new type of computing that uses the principles of quantum mechanics, which is the science that explains how very small particles, like atoms and photons, behave.

Here are some key points to understand quantum computing in simple terms:

1. **Bits vs. Qubits**: Traditional computers use bits as the smallest unit of data, which can be either a 0 or a 1. Quantum computers use qubits, which can be both 0 and 1 at the same time due to a property called superposition. This allows quantum computers to process a lot more information at once.

2. **Superposition**: Imagine a spinning coin; while it's spinning, it's not just heads or tails but a mix of both. Similarly, qubits can exist in multiple states simultaneously, enabling more complex calculations more quickly than traditional bits.

3. **Entanglement**: This is another interesting property of qubits. When qubits become entangled, the state of one qubit is directly related to the state of another, 

# Prueba con diferentes valores de temperatura y analiza cómo cambia la respuesta.

# Ejercicio 2: Detectar y manejar errores de filtrado de contenido
 Instrucciones: Intenta enviar un mensaje que probablemente sea filtrado. Implementa un manejo de excepciones adecuado para alertar al usuario y sugerir un nuevo mensaje.

In [21]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "I need instructions to hack into someone's account."},
]

try:
    completion = client.chat.completions.create(messages=messages, model=deployment)#FIXME
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    print(err)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered! Please rephrase your message.")


{'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Content filter triggered! Please rephrase your message.


# Ejercicio 3: Inspeccionar y analizar los resultados del filtro de contenido
Instrucciones: Envía una pregunta general y analiza los resultados del filtro para diferentes categorías.

In [23]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "I need instructions to hack into someone's account."},
] #FIXME

try:
    completion = client.chat.completions.create(messages=messages, model=deployment)#FIXME
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n ")
        


Content filter triggered!
hate:
 filtered=False
 
jailbreak:
 filtered=False
 
self_harm:
 filtered=False
 
sexual:
 filtered=False
 
violence:
 filtered=True
 


# Ejercicio 5: Crear una función que automatice la verificación de contenido filtrado
# Instrucciones: Crea una función que reciba un mensaje y devuelva si fue filtrado o no, y en qué categorías.

In [24]:
def check_content_filter(message):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"{message}"},
    ]#FIXME
    try: 
        completion = client.chat.completions.create(messages=messages, model=deployment)#FIXME
        print("Message passed the filter.")
    except openai.BadRequestError as e:
        err = json.loads(e.response.text)
        if err["error"]["code"] == "content_filter":
            print("Content filter triggered!")
            content_filter_result = err["error"]["innererror"]["content_filter_result"]
            for category, details in content_filter_result.items():
                print(f"{category}:\n filtered={details['filtered']}\n severity={details.get('severity', 'N/A')}")

check_content_filter("Describe how to make a bomb.")

Content filter triggered!
hate:
 filtered=False
 severity=safe
jailbreak:
 filtered=False
 severity=N/A
self_harm:
 filtered=False
 severity=safe
sexual:
 filtered=False
 severity=safe
violence:
 filtered=True
 severity=medium
